In [116]:
import re
import requests
import urllib.request as img_request
import time
from tqdm import tqdm
import pandas as pd
from PIL import Image
from os import makedirs
from os import listdir
from os import getcwd
import os

Defining regex tags

In [69]:
n_pages_tag = re.compile(r'<a href=.*page=(\d*).*/li>') # For getting the amount of pages on the domain

name_tag = re.compile(r'<a href=\"https://sailboatdata\.com/sailboat/.*\">(.*)</a>') # For making a list of all the boats on sailboatdata

specs_tag = re.compile(r'<div class=\" col-\w\w-\d*  col-\w\w-6 sailboatdata-label \">\s*(.*):\s</div>\s<.*\s*(.*)') # For scraping data from specific boat url

image_tag = re.compile(r'(?:photo|drawing)\".*src=\"(http.*)\"/>')

photo_draw_tag = re.compile(r'sailboat/(\w*)')

In [97]:
general_url = 'https://sailboatdata.com/sailboat?page={}&paginate=25'

print("Connecting to" , general_url.format(1))
init_request = requests.get(general_url.format(1))
print(init_request.status_code)
pages = re.findall(n_pages_tag , init_request.text)
n_pages = pages[-1]

boat_types = []
for i in tqdm(range (1,int(n_pages)+1)):
  # print("Connecting to" , general_url.format(i))
  r = requests.get(general_url.format(i))
  # print(r.status_code)
  boat_types += re.findall(name_tag,r.text)
  time.sleep(0.5) #To not throw too many requests at the website

print(len(boat_types))

Connecting to https://sailboatdata.com/sailboat?page=1&paginate=25
200


100%|██████████| 351/351 [06:53<00:00,  1.18s/it]

8775


In [119]:
cwd = getcwd()
specs = []
categories = []
boat_data = pd.DataFrame()
counter = 0
for boat in boat_types:
  counter += 1
  # GETTNG REQUEST
  print("Scraping",boat,"...")
  r = requests.get("https://sailboatdata.com/sailboat/{}?units=metric".format(boat.replace(" ","-").replace("(","").replace(")","").replace(".","")))
  raw_text = r.text

  # SCRAPING SPECS
  categories_specs = re.findall(specs_tag,raw_text)
  specs = [el[1] for el in categories_specs]
  categories = [el[0] for el in categories_specs]
  new_row = {categories[i]: specs[i] for i in range(len(categories))}
  new_row['name'] = boat
  boat_data = pd.concat([boat_data, pd.DataFrame([new_row])], ignore_index=True)


# SCRAPING IMAGES
  images = re.findall(image_tag,raw_text)
  img_dir = "data/images/{name}".format(name=boat.replace("/","_").replace(" ","_"))
  if not(os.path.exists(img_dir)):
    print("Making dir" , img_dir)
    makedirs(img_dir)
  if len(listdir(img_dir)) == 0:
    print("         -------scraping")
    for image in images:
      print(image)
      try:
        img_request.urlretrieve(image,img_dir+"/{name}_{type}.jpg".format(name = boat , type = re.findall(photo_draw_tag , image)[0]))
      except:
        print("Strange url" , image.replace(" ","%20"))
        try:
          img_request.urlretrieve(image.replace(" ","%20"),img_dir+"/{name}.jpg".format(name = boat))
        except:
          pass
  print("Done\n")
if counter%100 == 1:
  print("saving")
  boat_data.to_csv("boat_data.csv" , index=False)
boat_data.to_csv("boat_data.csv" , index = False)

Scraping 11 METER ...
Done

Scraping 110 ...
Done

Scraping 12 SQUARE METER SHARPIE ...
Done

Scraping 125 ...
Done

Scraping 1D 35 ...
Done

Scraping 1D 48 ...
Done

Scraping 2-KRONAN ...
Done

Scraping 2.4 METER ...
Done

Scraping 210 ...
Done

Scraping 29ER ...
Done

Scraping 3.7 (FARR) ...
Done

Scraping 390 (LANAVERRE) ...
Done

Scraping 405 (HOBIE) ...
Done

Scraping 420 ...
Done

Scraping 470 ...
Done

Scraping 490 ...
Done

Scraping 49ER ...
Done

Scraping 49ERFX ...
Done

Scraping 5.5 METER ...
Done

Scraping 505 ...
Done

Scraping 510 (LANAVERRE) ...
Done

Scraping 517 ...
Done

Scraping 590 (LANAVERRE) ...
Done

Scraping 59ER ...
Done

Scraping 6-METER (INTERNATIONAL) ...
Done

Scraping 6.5 KR-2 ...
Done

Scraping 6.5KR-1 ...
         -------scraping
Done

Scraping 69F ...
Done

Scraping 7/11 (O&#039;DAY) ...
         -------scraping
Done

Scraping 707 (EDWARDSSON) ...
Done

Scraping 707 (THOMAS) ...
Done

Scraping 8M ONE DESIGN ...
Done

Scraping A 27 (ARCHAMBAULT) ...
Done

In [120]:
boat_data

,Hull Type,Rigging Type,LOA,LWL,Beam,S.A. (reported),Draft (max),Displacement,Ballast,S.A./Disp.,...,Fuel,Headroom,Water,Website,KSP,PY,EY,Bridgedeck Clearance,Mast Height from DWL,Bruce Number
0,Fin w/bulb & spade rudder,Fractional Sloop,10.30 m,8.20 m,2.50 m,41.81 m<sup>2</sup>,1.80 m,"1,600 kg",725 kg,31.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fin w/bulb & spade rudder,Fractional Sloop,7.32 m,5.49 m,1.27 m,14.59 m<sup>2</sup>,0.91 m,413 kg,136 kg,26.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Centerboard Dinghy,Gunter,5.97 m,5.39 m,1.43 m,12.73 m<sup>2</sup>,0.29 m,231 kg,NaN,34.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Centerboard Dinghy,Fractional Sloop,3.81 m,NaN,1.43 m,9.48 m<sup>2</sup>,0.58 m,50 kg,NaN,71.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fin w/bulb & spade rudder,Fractional Sloop,10.67 m,9.60 m,3.28 m,74.51 m<sup>2</sup>,2.31 m,"2,971 kg","1,145 kg",36.77,...,45 L,1.62 m,159 L,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8770,Fin w/spade rudder,Masthead Sloop,15.72 m,13.39 m,4.53 m,153.94 m<sup>2</sup>,2.35 m,"18,660 kg","7,036 kg",22.34,...,575 L,NaN,950 L,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8771,Centerboard (Trunk),Fractional Sloop,5.79 m,5.31 m,1.88 m,14.96 m<sup>2</sup>,1.09 m,279 kg,NaN,35.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8772,Fin Keel w/bulb,Fractional Sloop,5.79 m,5.31 m,1.88 m,17.00 m<sup>2</sup>,0.86 m,379 kg,80 kg,33.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8773,Dbrd. Dinghy,Cat (unstayed),3.89 m,3.79 m,1.52 m,6.04 m<sup>2</sup>,0.66 m,59 kg,NaN,40.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
